## Lab 4, Exercise 2

In [2]:
import numpy as np
import pandas as pd

## Load data 


In [3]:
# Load the data in the following two CSVs:
# data/exercise2/lab4_normal_data.csv
# data/exercise2/lab4_malicious_data.csv
# The first consists completely of normal data, while the second consists completely of malicious data
# Note: Both sets of data contain the same features used in Exercise 1; the data has already been preprocessed
# (i.e., you can keep all the features and there are no labels in the CSVs)

# CODE HERE
normal_df = pd.read_csv('data/exercise2/lab4_normal_data.csv')
mal_df = pd.read_csv('data/exercise2/lab4_malicious_data.csv')

In [4]:
# Create 15 datasets, where the ith dataset consists of:
# - all normal data
# - only the ith malicious datapoint

# CODE HERE
dataframes = []
index_pred_map = {}
for y in range(normal_df.shape[0] + 1):
    index_pred_map[y] = 0

for i in range(15):
    mal_data = mal_df.iloc[[i]]
    df = pd.concat([mal_data, normal_df]).reset_index(drop=True)
    dataframes.append(df)

## Anomaly detection

In [27]:
# For each dataset, run isolation forests
#
# Use the following evaluation metric:
# - rank the anomalousness of each datapoint using the isolation forest
# - record the list index of each attack datapoint when sorting from most to least unusual
#     - e.g., if the attack datapoint is at index 0 in the list, we want to record the value 0
#
# Note: don't worry about ties in ranking
# Hint: What is the difference between isolation forest's 'decision_function' and 'predict' methods? 

# CODE HERE
from sklearn.ensemble import IsolationForest
avg_attack_rank = 0

for i in range(15):
    iso_forest = IsolationForest().fit(dataframes[i])
    labels = iso_forest.decision_function(dataframes[i])

    for y in range(len(labels)):
        index_pred_map[y] = labels[y]

    anomaly_rank = {k: v for k, v in sorted(index_pred_map.items(), key=lambda item: item[1])}
    attack_index = list(anomaly_rank.keys()).index(0)
    
    # Print this to see entire anomaly rankings
    # print(anomaly_rank, attack_index)
    print(f'Attack data anomaly ranking: {attack_index/len(labels)} ({attack_index} out of {len(labels)})')

    avg_attack_rank += attack_index/len(labels)
    # break

print(avg_attack_rank / 15)

Attack data anomaly ranking: 0.0025356690059106088 (142 out of 56001)
Attack data anomaly ranking: 0.3942429599471438 (22078 out of 56001)
Attack data anomaly ranking: 0.015231870859448939 (853 out of 56001)
Attack data anomaly ranking: 0.036981482473527255 (2071 out of 56001)
Attack data anomaly ranking: 0.005232049427688791 (293 out of 56001)
Attack data anomaly ranking: 0.475366511312298 (26621 out of 56001)
Attack data anomaly ranking: 0.3345118837163622 (18733 out of 56001)
Attack data anomaly ranking: 0.5681862823878145 (31819 out of 56001)
Attack data anomaly ranking: 0.013553329404832056 (759 out of 56001)
Attack data anomaly ranking: 0.5110980161068552 (28622 out of 56001)
Attack data anomaly ranking: 0.00044642059963214943 (25 out of 56001)
Attack data anomaly ranking: 0.001499973214764022 (84 out of 56001)
Attack data anomaly ranking: 0.015428295923287084 (864 out of 56001)
Attack data anomaly ranking: 0.016571132658345387 (928 out of 56001)
Attack data anomaly ranking: 0.00

## Questions:
1) Why is there no separate training and test set?

The isolation forest model is an unsupervised model that detects anomalies from a dataset and classifies the data as normal or an anomaly. This means that there is no separate training or test set there is assumed to be no clear labels to correct the model while training, so the training dataset is also used to test the model.

2) What is the metric measuring?  What would be a perfect score?  Bonus: What is the expected performance of an outlier detector that assigns a random score to each datapoint?

The metric measures how well the isolation forest can detect if the attack datapoint is an outsider compared to normal data. Ideally the perfect score for the attack data point is 0 or as close to 0 as possible. This means that the model recognizes the data point to be the most extreme point compared to the rest of the dataset.

The expected performance of a random detector would vary, but since there is only a single malicious datapoint, on average it may give it a score of 0.5. It is unlikely that the random score would be an extreme value (assuming that the randomness is uniform), thus the score for a single datapoint would average out to 0.5 after many scores (assuming from a scale of 0 to 1).

3) How well does the isolation forest perform compared to a perfect score? Bonus: How well does the isolation forest perform compared to a random detector?

On average, the isolation forest predicts the malicous datapoint as an outsider in the top 16% of the dataset. This means that on average, 16% of the dataset should be classified as an outsider or malicious datapoint including normal data, which means at least 16% of the predictions are false positives.

Compared to a random detector, if it assigns the malicious datapoint 0.5 on average, then 50% of the dataset would be false positives. The isolation forest performs better as it is less likely to report false positives.

4) What are some issues that would prevent this model from being practically deployed?

The isolation forest assumes that there is a small amount of outliers in a dataset that is otherwise normal behavior. This means that we assume that we know the percent of outliers within a dataset, which is not always true. For datasets where the number of malicious datapoints are a large portion of the dataset, if the malicious datapoints are similar enough, then it would become its own cluster rather than random outliers. This would inhibit the model and introduce more false negatives in the predictions.

5) What might happen if we inject five attack datapoints at a time?  What might happen if we inject 100 attack datapoints at a time?

If we inject 5 datapoints at a time, then the score for the attack data points would be closer to the average compared to injecting 1 datapoint. Similarly, if we inject 10 datapoints, then the scores for all those attack datapoints should also be closer to the average than the 5 attack injections. This is due to the dataset recognizing that the malicious datapoints are clustered near each other which forms its own forest. The model may not recognize malicious datapoints as outsiders compared to the entire dataset.

6) What is the effect of the parameters max_features and max_samples?  What other parameters could you adjust to change performance?

max_features is the number of feature to draw from the training dataset to train each base estimator. This affects the model to use a different number of features to learn the clusters of normal and malicious data. By default, all features are used, but we can instead use a subset of features to determine if a datapoint is malicious or not. If some features show no relationship between the different classes, then we may exclude those features.

max_samples is the number of samples to draw from the training dataset to train each base estimator. More samples means that the models learns from more datapoints which can potentially lead to overfitting. Currently, we use all datapoints to train the model.

We can change the number of estimators that the isolation forest uses. This can be used to increase or reduce the number of estimators or trees in the forest. Doing so may group other datapoints into a tree and improve the anomaly ranking of the malicious datapoint.

Bonus: What are some alternative anomaly detection models one could use instead of an isolation forest? Try one of these alternatives and compare performance.

Another anomally detection model is the local outlier factor where it scores datapoints based on how isolated the datapoint is with respect to its surrounding neighborhood. From the results, we see that it scores the malicious datapoints in the top 29% of all data when using 100 neighbors. This performs worse than isolation forests on average and has considerable variance in its rankings for different datapoints. Overall, isolation forests perform better than local outlier factor for this set of data.

In [17]:
from sklearn.neighbors import LocalOutlierFactor

avg_attack_rank = 0

for i in range(15):
    svm = LocalOutlierFactor(n_neighbors=100).fit(dataframes[i])
    labels = svm.negative_outlier_factor_

    for y in range(len(labels)):
        index_pred_map[y] = labels[y]

    anomaly_rank = {k: v for k, v in sorted(index_pred_map.items(), key=lambda item: item[1])}
    attack_index = list(anomaly_rank.keys()).index(0)
    
    # Print this to see entire anomaly rankings
    # print(anomaly_rank, attack_index)
    print(f'Attack data anomaly ranking: {attack_index/len(labels)} ({attack_index} out of {len(labels)})')

    avg_attack_rank += attack_index/len(labels)
    # break

print(avg_attack_rank / 15)


Attack data anomaly ranking: 0.22660309637327905 (12690 out of 56001)
Attack data anomaly ranking: 0.0035177943251013376 (197 out of 56001)
Attack data anomaly ranking: 0.9904287423438867 (55465 out of 56001)
Attack data anomaly ranking: 0.4223496008999839 (23652 out of 56001)
Attack data anomaly ranking: 0.16256852556204354 (9104 out of 56001)
Attack data anomaly ranking: 0.16258638238602882 (9105 out of 56001)
Attack data anomaly ranking: 0.24469205907037375 (13703 out of 56001)
Attack data anomaly ranking: 0.5052409778396815 (28294 out of 56001)
Attack data anomaly ranking: 0.21008553418688952 (11765 out of 56001)
Attack data anomaly ranking: 0.00301780325351333 (169 out of 56001)
Attack data anomaly ranking: 0.8238067177371833 (46134 out of 56001)
Attack data anomaly ranking: 0.11755147229513759 (6583 out of 56001)
Attack data anomaly ranking: 0.11755147229513759 (6583 out of 56001)
Attack data anomaly ranking: 0.19255013303333868 (10783 out of 56001)
Attack data anomaly ranking: 0